Install this required tensirflow_text library for BERT analysis
!pip install tensorflow_text

In [28]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text
import pandas as pd

Required URL's for BERT analysis

In [29]:
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

Reading the CSV file

In [30]:
data = pd.read_csv("Dataset/cleanedReviewsDateset100.csv")
data.head(5)

,product_id,page,page_order,recommended,date,bugType,haveBugs,text,hours,username,products,early_access,products_ismissing
0,1144200,1,2,True,01-01-22,na,0,love the game and would give it a 10 out 10 bu...,35.6,philcrest,31,True,False
1,108600,1,0,True,01-01-22,na,0,in what can only be described as a simplistic ...,6.5,ultramarine716,17,True,False
2,108600,1,7,False,01-01-22,invalid event occurance over time,1,fun game ruined by overly punishing update mor...,180.9,kobogen,157,True,False
3,108600,1,8,True,01-01-22,na,0,im a fan of 1st person graphics and a sucker f...,711.7,intorpere,93,True,False
4,1086940,1,0,True,01-01-22,na,0,great game in the making will be excessively e...,24.9,elkraf,46,True,False


Data preprocessing

In [31]:
X, y = (data['text'].values, data['haveBugs'].values)

Text data has to be integer encoded before feeding it into the RNN model. This can be easily achieved by using basic tools from the Keras library with only a few lines of code:

In [32]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tk = Tokenizer(lower = True)
tk.fit_on_texts(X)
X_seq = tk.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=100, padding='post')

Now, we split the data set into training and testing using sklearn’s train_test_split and keeping 25% of original data as a hold out set:

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size = 0.25, random_state = 1)

Furthemore, the training set can be split into training and validation set:

In [34]:
batch_size = 64
X_train1 = X_train[batch_size:]
y_train1 = y_train[batch_size:]
X_valid = X_train[:batch_size]
y_valid = y_train[:batch_size]

It is time to build the model and fit it on the training data using Keras:

In [35]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
vocabulary_size = len(tk.word_counts.keys())+1
max_words = 100
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

model.fit(X_train1,y_train1,validation_data=(X_valid,y_valid),batch_size=batch_size,epochs=10)
scores=model.evaluate(X_test,y_test,verbose=0)
print("Test Accuracy", scores[1])

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6927 - accuracy: 0.6000 - val_loss: 0.6871 - val_accuracy: 0.6094
Epoch 2/10
1/1 [==============================] - 0s 97ms/step - loss: 0.6883 - accuracy: 0.6000 - val_loss: 0.6816 - val_accuracy: 0.6094
Epoch 3/10
1/1 [==============================] - 0s 95ms/step - loss: 0.6837 - accuracy: 0.6000 - val_loss: 0.6752 - val_accuracy: 0.6094
Epoch 4/10
1/1 [==============================] - 0s 93ms/step - loss: 0.6783 - accuracy: 0.6000 - val_loss: 0.6669 - val_accuracy: 0.6094
Epoch 5/10
1/1 [==============================] - 0s 99ms/step - loss: 0.6719 - accuracy: 0.6000 - val_loss: 0.6556 - val_accuracy: 0.6094
Epoch 6/10
1/1 [==============================] - 0s 93ms/step - loss: 0.6652 - accuracy: 0.6000 - val_loss: 0.6467 - val_accuracy: 0.6094
Epoch 7/10
1/1 [==============================] - 0s 101ms/step - loss: 0.6716 - accuracy: 0.6000 - val_loss: 0.6486 - val_accuracy: 0.6094
Epoch 8/10
1/1 [============

In [36]:
# bert_preprocess_model = hub.KerasLayer(preprocess_url)
# text_test =data.text
# text_preprocessed = bert_preprocess_model(text_test)
# text_preprocessed.keys()
# text_preprocessed.values()